# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [101]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf

In [102]:
previsao_renda_df = pd.read_csv('previsao_de_renda.csv')
previsao_renda_df.sample(5)


,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
2156,2156,2015-03-01,14470,F,False,True,0,Empresário,Superior incompleto,Solteiro,Casa,30,1.353425,1.0,3488.05
13189,13189,2016-02-01,5846,M,True,True,0,Empresário,Superior completo,Solteiro,Casa,32,2.553425,1.0,8844.91
4333,4333,2015-05-01,3576,F,True,True,1,Pensionista,Secundário,Casado,Casa,56,NaN,3.0,1269.63
10925,10925,2015-11-01,1109,F,True,True,1,Servidor público,Superior completo,Viúvo,Casa,49,1.164384,2.0,3535.86
12015,12015,2016-01-01,6991,F,False,True,0,Pensionista,Superior incompleto,Viúvo,Casa,66,NaN,1.0,1466.61


In [103]:
previsao_renda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [104]:
previsao_renda_df.dropna(inplace=True)
previsao_renda_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

In [105]:
previsao_renda_df.drop(columns=['Unnamed: 0'], inplace=True)
previsao_renda_df.drop(columns=['data_ref'], inplace=True)
X = previsao_renda_df.drop(columns=['renda']).copy()
y = previsao_renda_df[['renda']]

In [106]:
previsao_renda_df.columns

Index(['id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda'],
      dtype='object')

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [107]:
previsao_renda_df["posse_de_veiculo"] = previsao_renda_df["posse_de_veiculo"].astype(int)
previsao_renda_df["posse_de_imovel"] = previsao_renda_df["posse_de_imovel"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

regularization_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

In [110]:
# Regularization
modelo = """renda ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda +
            educacao + estado_civil + tipo_residencia + idade + tempo_emprego + 
            C(qt_pessoas_residencia)"""
md = smf.ols(modelo, data = X_test)

reg = md.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt = 0  # com zero indica que é regularização do tipo Ridge
                        , alpha = 0.01)

reg.sumary()

# for alpha in regularization_alphas:
#     reg = md.fit_regularized(method = 'elastic_net' 
#                             , refit = True
#                             , L1_wt = 1
#                             , alpha = alpha)

#     print(f"O R-quadrado para Ridge com o Alpha iguail a {alpha} é {reg.rsquared}")
#     y_pred = reg.predict(X_test)

ValueError: endog has evaluated to an array with multiple columns that has shape (3107, 2). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).